<center>    
<h3>American Association of Physicists in Medicine</h3>    
<h3>Grand Challenge 2020</h3>
<h3>OpenKBP</h3>
<hr>
<h1>Introduction for Google Colab</h1>
<h3>January 25, 2022</h3>
</center>

# Setup

Before running this notebook, we need to get the repo which contains the data. The download should be quick as it's a sercer-to-server process.

In [1]:
# Setup

# Get the repo (only if not already cloned)
repo_dir = "open-kbp"
import os, sys
if not os.path.isdir(repo_dir):
    !git clone https://github.com/ababier/open-kbp.git {repo_dir}

# Add repo to path
sys.path.append(repo_dir)

# Install compatible ML stack for current Colab
!pip install -U pip setuptools wheel
!pip install -U "tensorflow==2.18.*" "keras==3.*" "numpy>=2.0"

# Install the remaining repo deps, skipping old pins
!grep -v -E "^(tensorflow|keras|numpy)== " open-kbp/requirements.txt > /tmp/req.txt
!pip install -r /tmp/req.txt


  Using cached numpy-2.4.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)
  Using cached black-22.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (52 kB)
  Using cached h5py-3.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached isort-5.11.4-py3-none-any.whl.metadata (13 kB)
  Using cached jupyter-1.0.0-py2.py3-none-any.whl.metadata (995 bytes)
  Using cached keras-2.11.0-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached matplotlib-3.6.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached more_itertools-9.0.0-py3-none-any.whl.metadata (31 kB)
  Using cached numpy-1.24.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached pylint-2.15.10-py3-none-any.whl.metadata (9.9 kB)
  Using cached scikit_learn-1.2.1-

In [2]:
# %tensorflow_version 2.x #  This ensures you use the newest version of tensorflow
%tensorflow_version 2.x # Use tensorflow 2

# Import provided classes and functions
import shutil
from pathlib import Path

from provided_code.data_loader import DataLoader
from provided_code.dose_evaluation_class import DoseEvaluator
from provided_code.network_functions import PredictionModel
from provided_code.utils import get_paths

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
!pip install -U pip setuptools wheel

# Keep TF 2.18 and keep NumPy compatible with TF 2.18
!pip install -U "numpy<2.1" "tensorflow==2.18.*" "keras==3.*"

# Install repo deps but skip old pins that break modern Colab
!grep -v -E '^(tensorflow|keras|numpy|h5py)=='
 /content/open-kbp/requirements.txt > /tmp/req.txt

!pip install -r /tmp/req.txt

The functions loaded from _provided\_code_ are written for this competition, and you can access them via the file
explorer on the left hand side of the Colab window. You're welcome to change them as much as
you'd like. If you use Google Drive now, keep in mind, however, that on Colab any changes you make to the files in your Google Drive will only be recognized by Colab when the _Runtime_ is restarted via the Restart

 Runtime option in the top toolbar. If you implement a neural network, we urge you to you start with the provided
 network architecture and network functions. The neural network we provide is only meant to be a template, and will not
 be a competitive model without some significant modifications.



# Data loading
Before we run anything, first define the paths where the provided data is stored and where the results (e.g., models, predictions) should be saved.

In [3]:
# Define project directories
primary_directory = Path(repo_dir).resolve()  # directory where everything is stored
provided_data_dir = primary_directory / "provided-data"
training_data_dir = provided_data_dir / "train-pats"
validation_data_dir = provided_data_dir / "validation-pats"
testing_data_dir = provided_data_dir / "test-pats"
results_dir = primary_directory / "results"  # where any data generated by this code (e.g., predictions, models) are stored

Name the model. This name will be used to label directories containing the results that the model generates. Also,
define how many epochs the model should be trained for. It will likely take a large number of epochs (e.g., 100-200)
to get good results.

In [4]:
test_time = False  # Only change this to True when the model has been fully tuned on the validation set
prediction_name = "baseline"  # Name model to train and number of epochs to train it for
num_epochs = 2

Retrieve the paths for all patient directories in the training set and seperate them into a list of paths for training
a model and another for hold-out testing.

In [5]:
# Prepare the data directory
training_plan_paths = get_paths(training_data_dir)  # gets the path of each plan's directory

# Model training

Initialize a data loader for the training set data, and use it to initialize a prediction model object. Call the
train_model method to train the model for the predefined number of epochs.

In [6]:
# Train a model
data_loader_train = DataLoader(training_plan_paths)
dose_prediction_model_train = PredictionModel(data_loader_train, results_dir, prediction_name,  "train")
dose_prediction_model_train.train_model(epochs=num_epochs, save_frequency=1, keep_model_history=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "generator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 128, 1)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 128, 10)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128, 128,  │          0 │ input_layer[0][0… │
│ (Concatenate)       │ 128, 11)          │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 64, 64,    │        704 │ concatenate[0][0] │
│                     │ 64, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 64, 64,    │          4 │ conv3d[0][0]      │
│ (BatchNormalizatio… │ 64, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 64, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_1 (Conv3D)   │ (None, 32, 32,    │        128 │ leaky_re_lu[0][0] │
│                     │ 32, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │          8 │ conv3d_1[0][0]    │
│ (BatchNormalizatio… │ 32, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 32, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 16, 16,    │        512 │ leaky_re_lu_1[0]… │
│                     │ 16, 4)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │         16 │ conv3d_2[0][0]    │
│ (BatchNormalizatio… │ 16, 4)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 16, 4)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 8, 8, 8,   │      2,048 │ leaky_re_lu_2[0]… │
│                     │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 8, 8,   │         32 │ conv3d_3[0][0]    │
│ (BatchNormalizatio… │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_3       │ (None, 8, 8, 8,   │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_4 (Conv3D)   │ (None, 4, 4, 4,   │      4,096 │ leaky_re_lu_3[0]… │
│                     │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 4, 4,   │         32 │ conv3d_4[0][0]  

 Total params: 29,561 (115.47 KB)

 Trainable params: 29,469 (115.11 KB)

...

Model loss at epoch 1 batch 97 is 1.532


99it [26:20, 15.67s/it]

Model loss at epoch 1 batch 98 is 1.533


100it [26:36, 15.96s/it]


Model loss at epoch 1 batch 99 is 1.534


Note that during training we will only keep models that are __save_frequency * keep_model_history__ epochs back from the
current epoch. We do this because models are very large (~1 GB).

Now that the model is trained we can use it to predict the dose for a set of hold-out patients from the validation or
testing set. The code block below gets the paths of all plans in the hold out set you selected earlier.


In [ ]:
# Define hold out set
hold_out_data_dir = validation_data_dir if test_time is False else testing_data_dir
stage_name, _ = hold_out_data_dir.stem.split("-")
hold_out_plan_paths = get_paths(hold_out_data_dir)

# Model testing

We start by making a new data loader for the held-out set, and use it to predict (and save) a
set of out-of-sample dose distributions. Note that we change the mode of the data loader to 'dose_prediction' to
load only the data needed to make a prediction.


In [8]:
# Predict dose for the held out set
data_loader_hold_out = DataLoader(hold_out_plan_paths)
dose_prediction_model_hold_out = PredictionModel(data_loader_hold_out, results_dir, model_name=prediction_name, stage=stage_name)
dose_prediction_model_hold_out.predict_dose(epoch=num_epochs)

Predicting dose with generator.


0it [00:00, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


1it [00:03,  3.15s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


...

38it [02:24,  3.44s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


39it [02:27,  3.29s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


40it [02:30,  3.77s/it]


Load each predicted dose distribution and evaluate it against the ground truth using the
competition metrics.

In [9]:
 # Evaluate dose metrics
data_loader_hold_out_eval = DataLoader(hold_out_plan_paths)
prediction_paths = get_paths(dose_prediction_model_hold_out.prediction_dir, extension="csv")
hold_out_prediction_loader = DataLoader(prediction_paths)
dose_evaluator = DoseEvaluator(data_loader_hold_out_eval, hold_out_prediction_loader)

# print out scores if data was left for a hold out set
if not data_loader_hold_out_eval.patient_paths:
    print("No patient information was given to calculate metrics")
else:
    dose_evaluator.evaluate()
    dvh_score, dose_score = dose_evaluator.get_scores()
    print(f"For this out-of-sample test on {stage_name}:\n\tthe DVH score is {dvh_score:.3f}\n\tthe dose score is {dose_score:.3f}")

40it [00:47,  1.19s/it]

For this out-of-sample test on validation:
	the DVH score is 25.492
	the dose score is 50.457


# Saving results

Once you're happy with your dose distributions you can zip up the predictions with the code block below. The zipped file
will contain the dose distributions for the validation set. It can be uploaded directly to CodaLab.

In [10]:
# Zip dose to submit
submission_dir = results_dir / "submissions"
submission_dir.mkdir(exist_ok=True)
submission_zipfile = shutil.make_archive(
    str(submission_dir / prediction_name),
    "zip",
    dose_prediction_model_hold_out.prediction_dir
)

You can save the model and submission to Google Drive or download them (on the left hand panel, click `...` -> `Download`).

In [ ]:
# Mount your personal google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Save the submission
submissions_on_drive = Path('/content/drive/MyDrive/open-kbp-subissions')
submissions_on_drive.mkdir(exist_ok=True)
shutil.copy(submission_zipfile, submissions_on_drive / f'{prediction_name}.zip')

> Note that the model will be lost once you close the Colab session. You can download or save what's in the `results` folder.